In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import trange

from tensorflow.keras import mixed_precision

### Test new detector class

In [2]:
# import
from detection.model import MaskRCNN

In [3]:
mask_model = MaskRCNN('models/mask_rcnn_inception_resnet_v2/saved_model/')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
# w, h = 1700, 1700
# fps = 10
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# writer = cv2.VideoWriter('Movie_1_result.mp4', fourcc, fps, (w, h))

# c = 0
# capture = cv2.VideoCapture('Movie_1.mkv')
# for i in trange(60*10 + 1):
#     ret, frame = capture.read()
#     if frame is None:
#         break
#     new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
#     for frame_coords in mask_model.detect(new_frame):
    
#         print(frame_coords)
#         if c >= 5:
#             break
#         c += 1
    
#     new_frame = draw_predictions(new_frame, raw_result)
    
#     writer.write(cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR))

# writer.release() 
    
# capture.release()

### Build service

In [5]:
from tracking.wrapper import DeepsortTracker

In [6]:
deep_sort_tracker = DeepsortTracker('tracking/model_data/mars-small128.pb')

In [7]:
from service import PigMonitoringService

In [8]:
pig_service = PigMonitoringService(mask_model, deep_sort_tracker)

In [9]:
def draw_predictions(frame, coords, colormap, track_mean_points):
    from config import visibility_zone, blind_zones, probability_threshold, mask_proba_threshold

    cv2.rectangle(frame, visibility_zone[0], visibility_zone[1], (0, 0, 255), 3)

    for point1, point2 in blind_zones:
        cv2.rectangle(frame, point1, point2, (255, 0, 0), 3)

    fontScale = 1
    thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX

    full_mask = np.zeros(frame.shape[:2])

    count = 0
    for coord in coords:
        color = colormap[coord.tracking_id]
        mask = coord.mask
        x_min, y_min, x_max, y_max = coord.x_min, coord.y_min, coord.x_max, coord.y_max

        full_mask[y_min:y_max, x_min:x_max] += mask
        
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 3)
        
        if len(track_mean_points[coord.tracking_id]) > 1:
            cv2.polylines(frame, np.array([track_mean_points[coord.tracking_id]]), False, color, thickness)

        org = (x_min, y_min)
        count += 1

    cv2.putText(frame, f'Pig count: {count}', (1300, 200), font, fontScale, (0, 255, 0), thickness, cv2.LINE_AA)

    full_mask = np.clip(full_mask, 0, 1).astype(np.uint8)

    frame = (frame * 0.75).astype(np.uint8)
    frame[:, :, 1] += full_mask * 64
    return frame

In [10]:
import random


def setColors_range(n_range: int):
    color_map = {i: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for i in range(n_range)}
    return color_map
        

In [11]:
import os
video_path = 'Movie_1.mkv'


track_mean_points = {}

colormap = setColors_range(150)

capture = cv2.VideoCapture(video_path)
length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fps = 10
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter(f'output/{os.path.basename(video_path)}', fourcc, fps, (width, height))

for _ in trange(length):
    ret, frame = capture.read()
    if frame is None:
        break
    new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    tracked_boxes = pig_service.process_frame(new_frame)
    
    for track_box in tracked_boxes:
        track_mean_points.setdefault(track_box.tracking_id, []).append([(track_box.x_min+track_box.x_max) // 2, (track_box.y_min+track_box.y_max) // 2])
    
    new_frame = draw_predictions(new_frame, tracked_boxes, colormap, track_mean_points)
    
    writer.write(cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR))

writer.release() 
    
capture.release()

 11%|█         | 64/601 [00:47<05:40,  1.58it/s] 

KeyboardInterrupt: 

In [ ]:
tracked_boxes

In [ ]:
frame = cv2.cvtColor(cv2.imread('test.png'), cv2.COLOR_BGR2RGB)

frame_boxes = mask_model.detect(frame)
bboxes = [i.bbox for i in frame_boxes]

In [ ]:
input_boxes

In [ ]:
tracked_boxes

In [ ]:
masks[18]

In [ ]:
len(tracked_boxes)

In [ ]:
tracked_boxes

In [2]:
import h5py
f = h5py.File('output/preds.h5', 'r')

In [17]:
a = f['Frame_10']['PolyMasks']

In [21]:
a['polymask_0'].value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """Entry point for launching an IPython kernel.


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)